In [103]:
import re
in_file = open("/Users/ales/laboratorij/prrorracun/final/posebni/ZR2008-005-II_2_POSEBNI_DEL_ZR_PRORACUNA_cleaned.csv")
out_file = open("/Users/ales/laboratorij/prrorracun/final/posebni/ZR2008-005-II_2_POSEBNI_DEL_ZR_PRORACUNA_processed.csv", "w")

In [104]:
fifo_lines = ["","",""]  # list should be of length 3

for line in in_file:
    incomplete = False
    # throw out useless stuff
    if re.match("^\"\",\"\",\"\(5=3/2\)\",\"\"", line):
        continue
    if re.match("^\"\",\"\",\"\(1\) \(2\) \(3\) \(4=3/1\)\",\"\"", line):
        continue
    if re.match("^\"\",\" Vir podatkov: Ministrstvo za finance, Direktorat.*", line):
        continue
    if re.match(".*v eurih.*", line):
        continue
    if re.match(".*PRORAČUNSKI UPORABNIKI\",\"Bilanca odhodkov.*", line):
        continue
    if re.match(".*SPU/PU/PPP/GPR/PPR/PP/K3\",\"Rebalans Veljavni.*", line):
        continue
    if re.match("^\"\",\"\",\"RE/VP\",\"\"", line):
        continue
        
    if re.match(".*OBRAZLOŽITEV POSEBNEGA DELA.*", line):
        beginning = re.search("^(.*) OBRAZLO", line).group(1)
        end = re.search("^.* OBRAZLOŽITEV POSEBNEGA DELA ZAKLJUČNEGA RAČUNA PRORAČUNA(.*)$",line).group(1)
        line = beginning + end
        
    if re.match(".*SPU/PU/PPP/GPR/PPR/PP/K3.*", line):
        beginning = re.search("^(.*) SPU/PU/PPP/GPR/PPR/PP/K3", line).group(1)
        end = re.search("^.* SPU/PU/PPP/GPR/PPR/PP/K3(.*)$",line).group(1)
        line = beginning + end     
        
        
    # Don't write incomplete rows
    if re.match("^\"[0-9]*\",\"\",\"[0-9]+.*", line):
        incomplete = True
    if re.match("^\"\",\".*", line):
        incomplete = True
    if re.match("^\"[0-9]*\",\".*\",\"[0-9]+\",\"\"", line):
        incomplete = True
    
    # repair broken rows
    fifo_lines[0] = fifo_lines[1]
    fifo_lines[1] = fifo_lines[2]
    fifo_lines[2] = line
    
    # first broken pattern
    # "121","","26135557 89,4 88,2",""
    # "","DRŽAVNI ZBOR","29228670 29646925",""
    if re.match("^\"[0-9]*\",\"\",\"[0-9]+.*", fifo_lines[1]) and \
       re.match("^\"\",\".*\",\"[0-9]* [0-9]*\",\"\"", line):
            # put the lines together
            incomplete = False
            beginning = re.search("^\"[0-9]*\",\"", fifo_lines[1]).group()
            middle = re.search("^\"\",\"([^\"]*\",\"[^\"]*)", line).group(1)
            end = re.search("^\"[0-9]*\",\"\",\"(.*\")", fifo_lines[1]).group(1)
            line = beginning + middle + " " + end
            
    # second broken pattern
    # "","","1873949 1790508 95,6 94,3",""
    # "3000","Plače","1899237",""
    if re.match("^\"\",\".*", fifo_lines[1]) and \
       re.match("^\"[0-9]*\",\"[^ ]+\",\"[0-9]+\",\"\"", line):
            # put the lines together
            incomplete = False
            beginning = re.search("^\"[0-9]*\",\"[^\"]*\",\"", line).group()
            middle_1 = re.search("^\"\",\"\",\"([0-9]* )", fifo_lines[1]).group(1)
            middle_2 = re.search("^\"[0-9]*\",\"[^\"]*\",\"([0-9]*)", line).group(1)
            end = re.search("^\"\",\"\",\"[0-9]*( [0-9]* .*\",\"\")$", fifo_lines[1]).group(1)
            line = beginning + middle_1 + middle_2 + end
    
    # third broken pattern
    # "","Stvarno premoženje - sredstva odškodnine","0 151 - 75,5",""
    # "8721","","200",""
    if re.match("^\"\",\".+\",\"[0-9]+.*\",\"\"$", fifo_lines[1]) and \
       re.match("^\"[0-9]*\",\"\",\"[0-9]*\",\"\"", line):
            # put the lines together
            incomplete = False
            beginning = re.search("^\"[0-9]*\",\"", line).group()
            middle_1 = re.search("^\"\",\"([^\"]*\",\"[0-9]* )", fifo_lines[1]).group(1)
            middle_2 = re.search("^\"[0-9]*\",\"\",\"([0-9]*)", line).group(1)
            end = re.search("^\"\",\"[^\"]*\",\"[0-9]*( [^\"]*\",\"\")", fifo_lines[1]).group(1)
            line = beginning + middle_1 + middle_2 + end
    
    # fourth broken pattern
    # "","Stvarno premoženje - sredstva od obremenitve s stavbno ali","0 0 - 0,0",""
    # "9554"," služnostno pravico","6480",""
    if re.match("^\"\",\".+\",\"[0-9]+.*\",\"\"$", fifo_lines[1]) and \
       re.match("^\"[0-9]*\",\" .+\",\"[0-9]*\",\"\"", line):
            # put the lines together
            incomplete = False
            beginning = re.search("^\"[0-9]*\",\"", line).group()
            middle_t1 = re.search("^\"\",\"([^\"]*)", fifo_lines[1]).group(1)
            middle_t2 = re.search("^\"[0-9]*\",\"( [^\"]+\",\")", line).group(1)
            middle_num1 = re.search("^\"\",\"[^\"]*\",\"([0-9]+ )", fifo_lines[1]).group(1)
            middle_num2 = re.search("^\"[0-9]*\",\" [^\"]+\",\"([0-9]+)", line).group(1)
            end = re.search("^\"\",\"[^\"]*\",\"[0-9]+( .*\",\"\")$", fifo_lines[1]).group(1)
            line = beginning + middle_t1 + middle_t2 + middle_num1 + middle_num2 + end
    
    if incomplete:
        continue
    # write to output
    print(line.rstrip(), file=out_file)

In [105]:
in_file.close()
out_file.close()